In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/constraint2021/data/consthindivalid.csv')
print(df['Post'])

df1 = pd.read_csv('/content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t1_consthindivalid.tsv',sep='\t')
df1 = df1.assign(multi_label = [labels for labels in df['Labels Set']])
df1.to_csv('/content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t1_consthindivalid.tsv',sep='\t',index=False)
# df1

* Uncleaned versions

In [10]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/constraint2021/data/uncleaned_train.csv')
# df = pd.read_excel('/content/drive/My Drive/constraint2021/data/uncleaned_train.xlsx')

df.rename(columns={'Post':'text','Labels Set':'multi_label'},inplace=True)
# df = df.assign(multi_label = [labels for labels in df['Labels Set']])
# df
df.isna().sum()
# df.to_csv('/content/drive/My Drive/constraint2021/data/siva/farm_uncleaned_train.tsv',sep='\t',index=False)
# df

Unique ID      0
text           0
multi_label    0
dtype: int64

#### Adding bin_label column

In [ ]:
df = df.assign(bin_label = [ 'not' if(label=='non-hostile') else 'host' for label in df['Labels Set'] ])
df
df.to_csv('/content/drive/My Drive/constraint2021/data/consthindivalid.csv',index=False)

####Check status

In [ ]:
import pandas as pd
# df1 = pd.read_csv('/content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t1_consthinditrain.tsv',sep='\t')
# df2 = df1[['text','multi_label']]
# df2.to_csv('/content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t2_consthinditrain.tsv',sep='\t',index=False)
df = pd.read_csv('/content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t2_consthinditrain.tsv',sep='\t')
df.columns

Index(['text', 'multi_label'], dtype='object')

###Tweet-pre-processing 

In [ ]:
!pip install tweet-preprocessor -q
!pip install emoji -q

#https://pypi.org/project/tweet-preprocessor/


     |████████████████████████████████| 51kB 4.5MB/s 


In [ ]:
import preprocessor as p
import emoji
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
punctuation_with_exceptions = list(string.punctuation)
punctuation_with_exceptions = [chr  for chr in punctuation_with_exceptions if(chr!='.' and chr!=',')]


p.set_options(p.OPT.NUMBER,p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG)

# Available Options: Option Name	Option Short Code URL	p.OPT.URL Mention	p.OPT.MENTION Hashtag	p.OPT.HASHTAG Reserved Words	p.OPT.RESERVED Emoji	p.OPT.EMOJI Smiley	p.OPT.SMILEY Number	p.OPT.NUMBER

def prepro_tweet(row):
    text_before_prepro = row['text']
    text_before_prepro = p.clean(text_before_prepro)
    return text_before_prepro

def deEmojify(row):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', row['text'])

def remove_RT(row):  # more of these symbols are created by emoji_to_text_before_prepro()
  text_before_prepro = row['text']
  text_before_prepro = text_before_prepro.replace('RT', ' ')
  return text_before_prepro


def punct_except_fullstop_comma(row):
  text_before_prepro = row['text']
  for char in punctuation_with_exceptions:
    text_before_prepro = text_before_prepro.replace(char, '')
  return text_before_prepro

def preprocess(df):
  df = df.assign(text = [ sent for sent in df['Post']])
  df['text'] = df.apply(prepro_tweet, axis=1)
  df['text'] = df.apply(remove_RT, axis=1)
  df['text'] = df.apply(punct_except_fullstop_comma, axis=1)
  df['text'] = df.apply(deEmojify,axis=1)
  return df


  

In [ ]:
import pandas as pd
# df = pd.read_csv('/content/drive/My Drive/constraint2021/data/consthindivalid.csv')
df = pd.read_csv('/content/drive/MyDrive/constraint2021/data/test.csv')

# print(df.columns)
print(df)
#EARLIER
# df1 = df[['Post','bin_label']]
# df1 = preprocess(df1)
# df1 = df1[['text','bin_label']]
# df1.to_csv('/content/drive/My Drive/constraint2021/data/cleaned_consthindivalid.csv',index=False)

#NOW
df1 = df[['Unique ID','Post']]
df1 = preprocess(df1)
print(df1)
df1 = df1[['Unique ID','text']]
df1.to_csv('/content/drive/My Drive/constraint2021/data/test_cleaned.csv',index=False)


# # df1.rename(columns={'text_before_prepro':'text'},inplace=True)
# df1
# df1.to_csv('/content/drive/My Drive/constraint2021/data/cleaned_consthindivalid.csv',index=False)

      Unique ID                                               Post
0             1  कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...
1             2  पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...
2             3  कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...
3             4  अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...
4             5  RT @_Pb_swain_: इन पंचर छापों को कोन समझाए कि ...
...         ...                                                ...
1648       1649  सोनू सूद से प्रेरणा लेकर आदिवासियों ने वो किया...
1649       1650  RT @ShilpiSinghINC: उमर खालिद पर तो UAPA लगा द...
1650       1651                           @RahulGandhi पप्पू कमीना
1651       1652  मदर टेरेसा ने अपना पूरा जीवन पीड़ितों की सेवा ...
1652       1653  आईपीएल में कोरोना का डर: रैना के हटने के बाद ह...

[1653 rows x 2 columns]
      Unique ID  ...                                               text
0             1  ...  कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...
1             2  ...  पटना 

In [ ]:
df1[-10:]

,Unique ID,text
1643,1644,ने सफलतापूर्वक का फ्लाइट टेस्ट किया
1644,1645,वैसे तो बीजेपी आईटी सेल से विरोधी दल ही परेशान...
1645,1646,"हरियाणा के शिक्षा, वन मंत्री कंवर पाल गुर्जर न..."
1646,1647,"दरभंगा एयरपोर्ट से जल्द ही उड़ान भरेंगे विमान,..."
1647,1648,हिंदू और मुस्लिमों के लिए अलगअलग वार्ड राज्य स...
1648,1649,सोनू सूद से प्रेरणा लेकर आदिवासियों ने वो किया...
1649,1650,उमर खालिद पर तो UAPA लगा दिया कपिल मिश्रा क...
1650,1651,पप्पू कमीना
1651,1652,मदर टेरेसा ने अपना पूरा जीवन पीड़ितों की सेवा ...
1652,1653,आईपीएल में कोरोना का डर रैना के हटने के बाद हर...


###To_farm

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/constraint2021/data/cleaned_consthindivalid.csv')
df
df.to_csv('/content/drive/My Drive/constraint2021/data/farm_cleaned_t1_consthindivalid.tsv',sep='\t',index=False)

In [ ]:
import pandas as pd
from collections import Counter
df = pd.read_csv('/content/drive/MyDrive/constraint2021/data/siva/farm_cleaned_t2_consthindivalid.tsv',sep='\t')
d = df[df['multi_label']!='non-hostile']
print(len(d))
# print(Counter(df['multi_label']))
Counter(d['multi_label'])

376


Counter({'defamation': 43,
         'defamation,fake': 4,
         'defamation,fake,hate': 1,
         'defamation,fake,hate,offensive': 1,
         'defamation,fake,offensive': 3,
         'defamation,hate': 10,
         'defamation,hate,offensive': 4,
         'defamation,offensive': 11,
         'fake': 144,
         'fake,hate': 3,
         'fake,offensive': 4,
         'hate': 68,
         'hate,offensive': 23,
         'offensive': 57})

In [ ]:
d.to_csv('/content/drive/MyDrive/constraint2021/data/siva/farm_cleaned_t2_consthindivalid_wo_non_hostile.tsv',sep='\t',index=False)

In [ ]:
a =  pd.read_csv('/content/drive/MyDrive/constraint2021/data/siva/farm_cleaned_t2_consthindivalid_wo_non-hostile.tsv',sep='\t')
a.columns

Index(['text', 'multi_label'], dtype='object')